In [387]:
from collections import abc, defaultdict, Counter
import random
import math
from functools import lru_cache

In [186]:
class DisjointSetsCollection:

    def __init__(self, iterable):
        it = iter(iterable)
        self.elements = {i:[i, 0] for i in it}

    def __getitem__(self, i):
        return self.elements[i]


In [287]:
def largest_component_size(A):
    G = {i:[] for i in A}
    N = len(A)
    for i in range(0, N):
        for j in range(i + 1, N):
            if gcd(A[i], A[j]) > 1:
                G[A[i]].append(A[j])
                G[A[j]].append(A[i])
    components = connected_components(G)
    # print(components)
    return max(map(len, components))


def gcd(a, b):
    if b == 0:
        return a
    return gcd(b, a % b)


def connected_components(G):
    visited = set()
    components = []
    for v in G:
        if v not in visited:
            c = dfs_visit(G, v)
            components.append(c)
            visited |= c
    return components


def dfs_visit(G, start):
    def _dfs(G, start, visited):
        for v in G[start]:
            if v not in visited:
                visited.add(v)
                visited |= _dfs(G, v, visited)
        return visited
    return _dfs(G, start, set())


In [388]:
# x : [parent, rank]
def _union(s, x, y):
    p1 = compress(s, x)
    p2 = compress(s, y)
    r1, r2 = s[p1], s[p2]
    if r1[0] != r2[0]:
        r1[1] += r2[1]
        r2[1] = 0
    r2[0] = r1[0]
    return r1[0]


def compress(s, x):
    node = s[x]
    if node[0] == x:
        return x
    val = node[1]
    node[0] = compress(s, node[0])
    s[node[0]][1] += val
    node[1] = 0
    return node[0]


def union(s, nodes):
    if len(nodes) < 2:
        return compress(s, nodes[0])
    x = nodes[0]
    for i in range(1, len(nodes)):
        x = _union(s, x, nodes[i])
    return s[x][0]


def primes_to(n):
    primes = [2, 3, 5]
    for i in range(7, n + 1, 2):
        isprime = True
        x = math.sqrt(i)
        for p in primes:
            if p > x:
                break
            isprime = bool(i % p)
            if not isprime:
                break
        if isprime:
            primes.append(i)
    return primes

PRIMES = primes_to(100_000)


def largest_component(A):
    primes = PRIMES[:bisect.bisect_left(PRIMES, max(A, default=0)) + 1]
    factors = {i:[i, 0] for i in primes}
    for a in set(A):
        divisors = []
        for p in primes:
            if p > a:
                break
            if a % p == 0:
                divisors.append(p)
        if divisors:
            factors[divisors[0]][1] += 1
            root = union(factors, divisors)
            # print('\nDivisors to merge:')
            # print(divisors)
            # print({k:v for k, v in factors.items() if v[1] > 0 or v[0] != k})
    res = max(factors.items(), key=lambda x: x[1][1])
    return res[1][1]

@lru_cache(100_000)
def prime_factors(n):
    result = []
    if n % 2 == 0:
        result.append(2)
        while n > 1 and n % 2 == 0:
            n = n // 2  
    while n > 1:
        for p in range(3, n + 1, 2):
            if n % p == 0:
                result.append(p)
                while n > 1 and n % p == 0:
                    n = n // p
                break
    return result


def largest_component_fast(A):
    factors = {}
    for a in set(A):
        divisors = prime_factors(a)
        for d in divisors:
            if d not in factors:
                factors[d] = [d, 0]
        if divisors:
            factors[divisors[0]][1] += 1
            root = union(factors, divisors)
            # print('\nDivisors to merge:')
            # print(divisors)
            # print({k:v for k, v in factors.items() if v[1] > 0 or v[0] != k})
    res = max(factors.items(), key=lambda x: x[1][1])
    return res[1][1]

In [368]:
prime_factors(2 * 3 * 5 * 7 * 11)

[2, 3, 5, 7, 11]

In [281]:
s = {1:[1, 10], 2:[1, 4], 5:[2, 10], 7:[5, 11]}
assert(compress(s, 7) == 1)


s = {1: [1, 10], 2:[2, 3], 3:[3, 1], 4:[4, 2]}
union(s, [1, 2])
assert(s == {1: [1, 13], 2:[1, 0], 3:[3, 1], 4:[4, 2]})
union(s, [3, 4])
assert(s == {1: [1, 13], 2:[1, 0], 3:[3, 3], 4:[3, 0]})
union(s, [2, 4])
assert(s == {1:[1, 16], 2:[1, 0], 3:[1, 0], 4:[3, 0]})
union(s, [4, 3])
assert(s == {1:[1, 16], 2:[1, 0], 3:[1, 0], 4:[1, 0]})
s[7] = [7, 1]
s[8] = [8, 1]
union(s, [3, 7])
assert(s == {1: [1, 17], 2: [1, 0], 3: [1, 0], 4: [1, 0], 7: [1, 0], 8: [8, 1]})
s[9] = [9, 11]
union(s, [8, 9])
assert(s == {1: [1, 17], 2: [1, 0], 3: [1, 0], 4: [1, 0], 7: [1, 0], 8: [8, 12], 9: [8, 0]})

s = {1:[1, 10], 2:[1, 2], 3:[2, 1], 4:[3, 7]}
assert(union(s, [4, 3]) == 1)
assert(s == {1:[1, 20], 2:[1, 0], 3:[1, 0], 4:[1, 0]})

s = {11:[11, 1], 29:[29, 1]}
assert(union(s, [11, 29]) == 11)
assert(s == {11: [11, 2], 29: [11, 0]})


In [119]:
largest_component_size([4, 6, 15, 35])
largest_component_size([20, 50, 9, 63])
largest_component_size([2,3,6,7,4,12,21,39])

[{35, 4, 6, 15}]
[{50, 20}, {9, 63}]
[{2, 3, 4, 6, 7, 39, 12, 21}]


8

**Tests**

In [375]:
faulty = []
for _ in range(100):
    for s in range(100, 110):
        arr = [random.randint(1, 100_000) for _ in range(s)]
        if largest_component(arr) != largest_component_size(arr):
            faulty.append(arr)

assert(not faulty)
assert(largest_component([74583, 83956, 56549, 6535, 25990, 64165, 93365, 60221, 72196, 88897, 38813, 62450, 55048, 12163, 33960, 10568, 39123, 75028, 19608, 76991, 95453, 25988, 19992, 70308, 45210, 14210, 43060, 2051, 41943, 89791, 32111, 40206, 67702, 25941, 56549, 76054]) == 28)

In [232]:
arrays = [[random.randint(1, 500) for _ in range(1000)] for _ in range(100)]

In [229]:
%timeit list(map(largest_component, arrays))

965 ms ± 1.14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [382]:
with open('testcase2.txt') as f:
    lines = f.readlines()

In [384]:
longlist2 = eval(''.join(lines))

In [ ]:
%load_ext line_profiler

In [309]:
%lprun -f largest_component largest_component(longlist)

Timer unit: 1e-07 s

Total time: 123.426 s
File: <ipython-input-285-5c2c080e8f2c>
Function: largest_component at line 51

Line #      Hits         Time  Per Hit   % Time  Line Contents
    51                                           def largest_component(A):
    52         1       3707.0   3707.0      0.0      primes = PRIMES[:bisect.bisect_left(PRIMES, max(A, default=0)) + 1]
    53         1      39370.0  39370.0      0.0      factors = {i:[i, 0] for i in primes}
    54      9515     141079.0     14.8      0.0      for a in set(A):
    55      9514     136983.0     14.4      0.0          divisors = []
    56  48230667  382129320.0      7.9     31.0          for p in primes:
    57  48230666  406862901.0      8.4     33.0              if p > a:
    58      9513     146762.0     15.4      0.0                  break
    59  48221153  441497217.0      9.2     35.8              if a % p == 0:
    60     25472     392957.0     15.4      0.0                  divisors.append(p)
    61      

In [373]:
%lprun -f largest_component_fast largest_component_fast(longlist)


Timer unit: 1e-07 s

Total time: 17.2314 s
File: <ipython-input-369-0257df48d6c5>
Function: largest_component_fast at line 87

Line #      Hits         Time  Per Hit   % Time  Line Contents
    87                                           def largest_component_fast(A):
    88         1    7671647.0 7671647.0      4.5      primes = primes_to(max(A, default=0))
    89         1         23.0     23.0      0.0      factors = {}
    90         1          9.0      9.0      0.0      divisors = []
    91      9515      81141.0      8.5      0.0      for a in set(A):
    92      9514  162329003.0  17062.1     94.2          divisors = prime_factors(a)
    93     34986     313598.0      9.0      0.2          for d in divisors:
    94     25472     201806.0      7.9      0.1              if d not in factors:
    95      2583      37246.0     14.4      0.0                  factors[d] = [d, 0]
    96      9514      71967.0      7.6      0.0          if divisors:
    97      9514     129185.0     13.

In [336]:
%timeit largest_component_fast(longlist)

4.52 s ± 110 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [337]:
%timeit largest_component(longlist)

4.4 s ± 524 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [351]:
%timeit Solution().largestComponentSize(arrays[16])

9.82 ms ± 161 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [372]:
%timeit largest_component(longlist)

4.18 s ± 24.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [377]:
%timeit largest_component_fast(longlist2)

2.75 s ± 4.66 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [389]:
%timeit largest_component_fast(longlist2)

82 ms ± 898 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [394]:
from collections import deque
class Solution:  # Sieve + BFS 
    
    def largestComponentSize(self, A) -> int:
        
        def sieve():
            i = 2
            while i < len(spf):
                spf[i] = 2
                i += 2
            i = 3
            while i * i < len(spf):
                if spf[i] == i:
                    j = i
                    while j < len(spf):
                        if spf[j] == j:
                            spf[j] = i
                        j += i
                i += 2
        
        spf, di, factors, visited_nums, visited_factors, ans = [i for i in range(max(A) + 1)], defaultdict(set), defaultdict(set), set(), set(), 1
        
        sieve()  # 1
        
        for num in A:
            x = num
            while x != 1:
                di[spf[x]].add(num)
                factors[num].add(spf[x])
                x //= spf[x]
        
        for num in A:  # 2
            if num in visited_nums:
                continue
                
            visited_nums.add(num)
            cur, queue = 1, deque([])
            for factor in factors[num]:
                queue.append(factor)
                
            while queue: 
                factor = queue.popleft()
                visited_factors.add(factor)
                for next_num in di[factor]:  # 3
                    if next_num in visited_nums:
                        continue
                        
                    visited_nums.add(next_num)
                    cur += 1
                    for next_factor in factors[next_num]:
                        if next_factor in visited_factors:
                            continue
                        
                        visited_factors.add(next_factor)
                        queue.append(next_factor)
                
            ans = max(ans, cur)
        
        return ans

In [399]:
Solution().largestComponentSize(longlist)

8798

In [397]:
len(longlist2)

18176

In [400]:
90000 ** 0.5

300.0

In [401]:
math.log2(90000)

16.457637380991763

In [403]:
class Solution:
    def __init__(self):
        spf = list(range(100_001))
        for i in range(4, 100_001, 2):
            spf[i] = 2
        for i in range(3, 100_001, 2):
            if spf[i] == i:
                for j in range(i * i, 100_001, i):
                    if spf[j] == j:
                        spf[j] = i
        self.spf = spf
        
    def getFactors(self, n):
        res = []
        spf = self.spf
        while n > 1:
            p = spf[n]
            res.append(p)
            n = n // p
        return res
    
    
    def largestComponentSize(self, A) -> int:
        if not A:
            return 0
        factors = {}
        divisors = []
        for a in set(A):
            divisors = self.getFactors(a)
            for d in divisors:
                if d not in factors:
                    factors[d] = [d, 0]
            if divisors:
                factors[divisors[0]][1] += 1
                root = union(factors, divisors)
        if not factors:
            return 1
        res = max(factors.items(), key=lambda x: x[1][1])
        return res[1][1]
    
    
def _union(s, x, y):
    p1 = compress(s, x)
    p2 = compress(s, y)
    r1, r2 = s[p1], s[p2]
    if r1[0] != r2[0]:
        r1[1] += r2[1]
        r2[1] = 0
    r2[0] = r1[0]
    return r1[0]


def compress(s, x):
    node = s[x]
    if node[0] == x:
        return x
    val = node[1]
    node[0] = compress(s, node[0])
    s[node[0]][1] += val
    node[1] = 0
    return node[0]


def union(s, nodes):
    if len(nodes) < 2:
        return compress(s, nodes[0])
    x = nodes[0]
    for i in range(1, len(nodes)):
        x = _union(s, x, nodes[i])
    return s[x][0]

In [406]:
Solution().largestComponentSize(longlist)

8798

In [407]:
faulty = []
solution = Solution()
for _ in range(100):
    for s in range(100, 110):
        arr = [random.randint(1, 100_000) for _ in range(s)]
        if solution.largestComponentSize(arr) != largest_component_fast(arr):
            faulty.append(arr)

assert(not faulty)
assert(largest_component([74583, 83956, 56549, 6535, 25990, 64165, 93365, 60221, 72196, 88897, 38813, 62450, 55048, 12163, 33960, 10568, 39123, 75028, 19608, 76991, 95453, 25988, 19992, 70308, 45210, 14210, 43060, 2051, 41943, 89791, 32111, 40206, 67702, 25941, 56549, 76054]) == 28)

In [408]:
def choose_between(nums, a, b):
    for i in range(b, a - 1, -1):
        if i in nums:
            del nums[nums.index(i)]
            return i
    return -1

In [409]:
A = [2, 0, 0, 1]

In [411]:
choose_between(A, 0, 2)

2

In [412]:
A

[0, 0, 1]

In [413]:
choose_between(A, 0, 3)

1

In [414]:
A

[0, 0]

In [415]:
choose_between(A, 0, 5)

0

In [416]:

A

[0]

In [417]:
choose_between(A, 0, 9)

0

In [418]:
A

[]